In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/MNIST", one_hot=True)

In [ ]:
image_size = mnist.train.images.shape[1]
image_height = int(np.sqrt(image_size))
image_width = int(np.sqrt(image_size))
print("{:d} = {:d} x {:d}".format(image_size, image_height, image_width))

In [ ]:
labels_size = mnist.train.labels.shape[1]
print(labels_size)

In [ ]:
def weight_variable(name, shape):
    return tf.get_variable(name, shape, dtype=np.float32,
        initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode="FAN_IN", uniform=True))

In [ ]:
def bias_variable(name, shape):
    return tf.get_variable(name, shape, dtype=np.float32, initializer=tf.zeros_initializer())

In [ ]:
def conv2d(name, t_input, patch, input_channels, output_channels):
    with tf.variable_scope(name):
        t_weight = weight_variable("weight", [patch, patch, input_channels, output_channels])
        t_conv = tf.nn.conv2d(t_input, t_weight, strides=[1, 1, 1, 1], padding="VALID", name="conv")
        t_bias = bias_variable("bias", [output_channels])
        t_linear = tf.add(t_conv, t_bias, name="linear")
        t_activation = tf.nn.tanh(t_linear, name="activation")
    return t_activation

In [ ]:
def max_pool_2x2(name, x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name=name)

In [ ]:
def fc(name, t_input, input_size, hidden_size, activation=True):
    with tf.variable_scope(name):
        t_weight = weight_variable("weight", [input_size, hidden_size])
        t_matmul = tf.matmul(t_input, t_weight, name="matmul")
        t_bias = bias_variable("bias", [hidden_size])
        t_linear = tf.add(t_matmul, t_bias, name="linear")
        if activation:
            return tf.nn.tanh(t_linear, name="activation")
        else:
            return t_linear

In [ ]:
image_channels = 1
conv1_channels = 6
conv1_patch = 5
conv2_channels = 16
conv2_patch = 5
fc_input = conv2_channels * conv2_patch * conv2_patch
fc1_hidden = 120
fc2_hidden = 84

learning_rate = 0.01

tf.reset_default_graph()

t_flat_images = tf.placeholder(tf.float32, shape=[None, image_size], name="flat_images")
t_labels = tf.placeholder(tf.float32, shape=[None, labels_size], name="labels")

t_square_images = tf.reshape(t_flat_images, [-1, image_height, image_width, image_channels], name="square_images")
t_padded_images = tf.pad(t_square_images, [[0, 0], [2, 2], [2, 2], [0, 0]], name="padded_images")

t_conv1 = conv2d("conv1", t_padded_images, conv1_patch, image_channels, conv1_channels)

t_sampling1 = max_pool_2x2("sampling1", t_conv1)

t_conv2 = conv2d("conv2", t_sampling1, conv2_patch, conv1_channels, conv2_channels)

t_sampling2 = max_pool_2x2("sampling2", t_conv2)

t_fc_input = tf.reshape(t_sampling2, [-1, fc_input], name="fc_input")

t_fc1 = fc("fc1", t_fc_input, fc_input, fc1_hidden)

t_fc2 = fc("fc2", t_fc1, fc1_hidden, fc2_hidden)

t_logits = fc("fc3", t_fc2, fc2_hidden, labels_size, activation=False)
t_predictions = tf.nn.softmax(t_logits, name="predictions")

t_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=t_logits, labels=t_labels))
t_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(t_loss)

t_correct = tf.reduce_sum(
    tf.cast(
        tf.equal(tf.argmax(t_predictions, 1), tf.argmax(t_labels, 1)),
        tf.float32
    )
)

In [ ]:
batch_size = 128
log_every_batches = 100

def run_batches(batches, train=True):
    epoch_loss = 0.0
    epoch_correct = 0.0
    
    iterations = batches.num_examples // batch_size

    for iteration in range(iterations):
        batch = batches.next_batch(batch_size)

        if train:
            sess.run(t_optimizer, feed_dict={t_flat_images: batch[0], t_labels: batch[1]})
            
        loss, correct = sess.run([t_loss, t_correct],
                                 feed_dict={t_flat_images: batch[0], t_labels: batch[1]})
        
        epoch_loss += loss
        epoch_correct += correct
        
        if train and iteration % log_every_batches == log_every_batches - 1:
            print("Train Batch: {:5d} Loss: {:.4f}".format(iteration + 1, loss))

    return epoch_loss / iterations, epoch_correct / (iterations * batch_size)

In [ ]:
epochs = 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_id in range(epochs):
        train_loss, train_accuracy = run_batches(mnist.train, train=True)
        test_loss, test_accuracy = run_batches(mnist.test, train=False)

        print("Epoch: {:5d} Train Loss: {:.4f} Test Loss: {:.4f} Train Accuracy: {:.4f} Test Accuracy: {:.4f}".format(
            epoch_id + 1, train_loss, test_loss, train_accuracy, test_accuracy))